## Create accumulation curves
- One curve in one plot
- Multiple curves in 1 plot
- Change richness into somehtin thats not italic

In [ ]:
# load R packages 
library(vegan)
library(dplyr)
library(tidyr)
library(tidyverse)

In [ ]:
# one curve one plot
# Load in the OTU data (this table contains integers instead of values with a comma)
otu1 <- read.table("./csv_out/mags.all.csv", sep=",", header=T, row.names=1)

# transpose (rows=OTUs, columns=sample)
otu <- t(otu1)
sp.all <- specaccum(otu, method = "random", permutations = 100)

# Extract each of the permutations and reformat
perm <- sp.all$perm

perm.tidy <- as.tibble(perm) %>% 
  mutate(Sites = 1:nrow(.)) %>% 
  gather(key = "Permutation", value = "Species", -Sites) 

# Extract the consensus richness per number of sites sampled and reformat
richness <- data.frame(Sites = sp.all$sites, Species = sp.all$richness)

# create the actual plot
pdf('accumplot_all.mag.pdf')
ggplot(perm.tidy, aes(Sites, Species)) +
  geom_point(alpha = 0.3, shape = 1, size = 2, color='#2a9d8f') +
  #scale_color_brewer(palette = my_palette) +
  geom_line(data = richness, size = 1) +
  labs(y= "# MAGs", x = "Samples") +
  theme_bw() +
  theme(text = element_text(size = 18))
dev.off()

In [ ]:
# multiple curves in 1 plot (can probs write a function but idk how to create functions in R)
otu_all1 <- read.table("./csv_out/mags.all.csv", sep=",", header=T, row.names=1)
otu_991 <- read.table("./csv_out/mags.99.csv", sep=",", header=T, row.names=1)
otu_951 <- read.table("./csv_out/mags.95.csv", sep=",", header=T, row.names=1)

# transpose
otu_all <- t(otu_all1)
otu_99 <- t(otu_991)
otu_95 <- t(otu_951)

# accumulation curves
sp.all <- specaccum(otu_all, method = "random", permutations = 100)
sp.99 <- specaccum(otu_99, method = "random", permutations = 100)
sp.95 <- specaccum(otu_95, method = "random", permutations = 100)


In [ ]:
# Extract permutations and reformat
perm <- sp.all$perm

perm.tidy <- as.tibble(perm) %>% 
  mutate(Sites = 1:nrow(.)) %>% 
  gather(key = "Permutation", value = "Species", -Sites) 

# Extract the consensus richness per number of sites sampled and reformat
richness <- data.frame(Sites = sp.all$sites, Species=sp.all$richness)

# Extract permutations and reformat
perm.99 <- sp.99$perm

perm.99.tidy <- as.tibble(perm.99) %>% 
  mutate(Sites = 1:nrow(.)) %>% 
  gather(key = "Permutation", value = "Species", -Sites) 

# Extract the consensus richness per number of sites sampled and reformat
richness.99 <- data.frame(Sites = sp.99$sites, Species=sp.99$richness)

# Extract permutations and reformat
perm.95 <- sp.95$perm

perm.95.tidy <- as.tibble(perm.95) %>% 
  mutate(Sites = 1:nrow(.)) %>% 
  gather(key = "Permutation", value = "Species", -Sites) 

# Extract the consensus richness per number of sites sampled and reformat
richness.95 <- data.frame(Sites = sp.95$sites, Species=sp.95$richness)

In [ ]:
# combine results 
all.perm  <- rbind(mutate(perm.tidy, Extraction = "all MAGs"),
                  mutate(perm.99.tidy, Extraction = "99% ANI clust"),
                   mutate(perm.95.tidy, Extraction = "95% ANI clust"))
all.richness  <- rbind(mutate(richness, Extraction = "all MAGs"),
                     mutate(richness.99, Extraction = "99% ANI clust"),
                      mutate(richness.95, Extraction = "95% ANI clust"))

In [ ]:
# create the actual plot
pdf('accumplot_all_3_clust.pdf')
ggplot(all.perm, aes(Sites, Species, color = Extraction)) +
  geom_point(alpha = 0.3, shape = 1, size = 2) +
  scale_color_manual(values = c("#2a9d8f", "#e9c46a", '#e76f51')) + 
  geom_line(data = all.richness, size = 1) +
  labs(y= "# MAGs", x = "Samples") +
  theme_bw() +
  theme(text = element_text(size = 18))
dev.off()
